#### Berkeley CS 285 - Deep Reinforcement Learning, Decision Making, and Control - Fall 2020

# Assignment 1: Imitation Learning

##### All pictures and slides are from Sergei Levine's course CS285 - Deep RL

> http://rail.eecs.berkeley.edu/deeprlcourse/

*The goal of this assignment is to experiment with imitation learning, including direct behavior cloning and
the DAgger algorithm. In lieu of a human demonstrator, demonstrations will be provided via an expert policy
that we have trained for you. Your goals will be to set up behavior cloning and DAgger, and compare their
performance on a few different continuous control tasks from the OpenAI Gym benchmark suite. Turn in your
report and code as described in Section 4.*

*The starter-code for this assignment can be found at*

> https://github.com/berkeleydeeprlcourse/homework_fall2020

*You have the option of running the code either on Google Colab or on your own machine. Please refer to the
README for more information on setup.*

# Wide screen notebook

Do not run if you prefer the default setup.

In [21]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

TODO:
    
   * Estructura del programa/sistemita
   * Solucion de cada parte que habia que solucionar
   * Analisis que píde el ejercicio
    

# Intro 

# Supervised Learning of Behaviours

We are going to use the same Supervised Learning setup as a neural network learning to map inputs to output category (eg. Cat/Dog photos classifier), but instead of learning to classify pictures, we're going to learn ("clone") behaviours (as a sequence of decisions).

We can think of this as a "copy-cat" of a given expert behaviour, from which we want to learn to imitate as close as possible.

![](./img/terminology.png)

The problem can be formalized as a probabilistic graphical model, where each node correspond to taking an action, being an state from which we only "see" an observation and "moving" to a new state.

* **State:** True configuration of the system (eg. All physical variables of a ball falling)

* **Observation:** Results from the state. Maybe NOT contain the whole information of the state (eg. A photo of the ball falling)

![](./img/observation_and_state.png)

![](./img/imitation.png)

Using only one photo (per timestep) as input **DOES NOT work** right away.

#### Problem

Small mistakes get accumulated until the observed state is very different from the demostrator behaviour, resulting in very bad performance (because the states are unknown for the agent).

#### One solution

Use 3 photos: a centered one, and a diagonally pointing to the left/right, each labeled as "go forth", "rotate (compensate) right" and "rotate left" 

![](./img/three_input.png)

# DAgger: Dataset Aggregation

![](./img/dagger.png)

In this homework, we're going to change step 3:

> Instead of asking a human, we're going to load an expert trained neural network, and sample an action given an observation.

This happens inside `rl_trainer.py` on the method `do_relabel_with_expert(...)`

```python
    def do_relabel_with_expert(self, expert_policy, paths):
        print("Relabelling collected observations with labels from an expert policy...")

        # TODO relabel collected obsevations (from our policy) with labels from an expert policy
        # HINT: query the policy (using the get_action function) with paths[i]["observation"]
        # and replace paths[i]["action"] with these expert labels
        for i in range(len(paths)):
            obs = paths[i]["observation"]
            expert_action = expert_policy.get_action(obs)
            paths[i]["action"] = expert_action
        return paths
```

Where the returned `paths` is a list of dictionaries containing the information for the trajectories (s,a,r,s') (see utils.py to find a Path object).

# Code

As with the `do_relabel_with_expert` method, lets go over each TODO piece of code.

The files in order to read and to be edited are:

1. `scripts/run_hw1.py` (read-only file)

2. `infrastructure/rl_trainer.py`

3. `agents/bc_agent.py` (read-only file)

4. `policies/MLP_policy.py`

5. `infrastructure/replay_buffer.py`

6. `infrastructure/utils.py`

7. `infrastructure/pytorch_util.py`

## 1. `scripts/run_hw1.py` (read-only file)

You can read all possible parameters to get a better idea of the algorithms to be implemented.

Also you can edit the main method so you don't need to write the parameters in the console, but directly run the file, eg:

```python
if __name__ == "__main__":
    args = ['--expert_policy_file', '/home/user/CS_285-Deep_Reinforcement_Learning/hw1/cs285/policies/experts/Ant.pkl',
            '--expert_data', '/home/user/CS_285-Deep_Reinforcement_Learning/hw1/cs285/expert_data/expert_data_Ant-v2.pkl',
            '--env_name', 'Ant-v2',
            '--exp_name', 'bc_ant',
            '--ep_len', '5000',
            '--eval_batch_size', '5000',
            '--train_batch_size', '1000',
            '--num_agent_train_steps_per_iter', '100',
            '--no_gpu'
            ]
    main(args)
```

Later on you can automate an hyperparameter search algorithms that calls main in a similar way.

## 2. `infrastructure/rl_trainer.py`

**Note**: This one uses `utils.sample_trajectories` and `utils.sample_n_trajectories`, so maybe you can read them right away to better understand whats going on here.

**Note:** Added tqdm() to train_agent()


In [22]:
!pygmentize cs285/infrastructure/rl_trainer.py

from collections import OrderedDict
import numpy as np
import time
import pickle
from tqdm import tqdm
import pkbar
import json

import gym
import torch

from cs285.infrastructure import pytorch_util as ptu
from cs285.infrastructure.logger import Logger
from cs285.infrastructure import utils

# how many rollouts to save as videos to tensorboard
MAX_NVIDEO = 2
MAX_VIDEO_LEN = 40  # we overwrite this in the code below


class RL_Trainer(object):

    def __init__(self, params):

        #############
        ## INIT
        #############

        # Get params, create logger, create TF session
        self.params = params
        self.logger = Logger(self.params['logdir'])

        # Set random seeds
        seed = self.params['seed']
        np.random.seed(seed)
        torch.manual_seed(seed)
        ptu.init_gpu(
            use_gpu=not self.params['no_gpu'],
            gpu_id=self.params['which_gpu']
        )

        #############
        ## ENV
        #############

        # Mak

And inside `perform_logging`, added a call to a method to save recorded statistics to a file (to open later and compare runs).

```python
            # perform the logging
            for key, value in logs.items():
                print('{} : {}'.format(key, value))
                self.logger.log_scalar(value, key, itr)
                # TODO: call to log values for matplotlib or similar
                self.logs_to_file(logs)
            print('Done logging...')
            self.logger.flush()
```

## 6. `infrastructure/utils.py`

Here are defined the methods used before.

They "sample trajectories" ie. simulate and record T trajectories $\tau = [ (s_1, a_1, r_1, s'_1), (s_2, a_2, r_2, s'_2), \dots , (s_T, a_T, r_T, s'_T) ]$

In [23]:
!pygmentize cs285/infrastructure/utils.py

import numpy as np
import time

############################################
############################################

def sample_trajectory(env, policy, max_path_length, render=False, render_mode=('rgb_array')):
    # initialize env for the beginning of a new rollout
    ob = env.reset() #TODO # HINT: should be the output of resetting the env

    # init vars
    obs, acs, rewards, next_obs, terminals, image_obs = [], [], [], [], [], []
    steps = 0
    while True:

        # render image of the simulated env
        if render:
            if 'rgb_array' in render_mode:
                if hasattr(env, 'sim'):
                    image_obs.append(env.sim.render(camera_name='track', height=500, width=500)[::-1])
                else:
                    image_obs.append(env.render(mode=render_mode))
            if 'human' in render_mode:
                env.render(mode=render_mode)
                time.sleep(env.model.opt.timestep)

        # use the most recent ob to decide what t

## 3. `agents/bc_agent.py` (read-only file)

BC (as in Behaviour Clonning) Agent is a class that represents the agent entity.

It's initialized with an random initialized network (see MLPPolicySL) an empty replay buffer, to train using Supervised Learning on top of expert data.

In [24]:
!pygmentize cs285/agents/bc_agent.py

from cs285.infrastructure.replay_buffer import ReplayBuffer
from cs285.policies.MLP_policy import MLPPolicySL
from .base_agent import BaseAgent


class BCAgent(BaseAgent):
    def __init__(self, env, agent_params):
        super(BCAgent, self).__init__()

        # init vars
        self.env = env
        self.agent_params = agent_params

        # actor/policy
        self.actor = MLPPolicySL(
            self.agent_params['ac_dim'],
            self.agent_params['ob_dim'],
            self.agent_params['n_layers'],
            self.agent_params['size'],
            discrete=self.agent_params['discrete'],
            learning_rate=self.agent_params['learning_rate'],
        )

        # replay buffer
        self.replay_buffer = ReplayBuffer(self.agent_params['max_replay_buffer_size'])

    def train(self, ob_no, ac_na, re_n, next_ob_no, terminal_n):
        # training a BC agent refers to updating its actor using
        # the given observations and corresponding action labels
      

## 4. `policies/MLP_policy.py`

Note the differences and similitudes between discrete and continuous policy.

MLP: Multi Layer Perceptron

In [25]:
!pygmentize cs285/policies/MLP_policy.py

import abc
import itertools
from typing import Any
from torch import nn
from torch.nn import functional as F
from torch import optim

import numpy as np
import torch
from torch import distributions

from cs285.infrastructure import pytorch_util as ptu
from cs285.policies.base_policy import BasePolicy


class MLPPolicy(BasePolicy, nn.Module, metaclass=abc.ABCMeta):

    def __init__(self,
                 ac_dim,
                 ob_dim,
                 n_layers,
                 size,
                 discrete=False,
                 learning_rate=1e-4,
                 training=True,
                 nn_baseline=False,
                 **kwargs
                 ):
        super().__init__(**kwargs)

        # init vars
        self.ac_dim = ac_dim
        self.ob_dim = ob_dim
        self.n_layers = n_layers
        self.discrete = discrete
        self.size = size
        self.learning_rate = learning_rate
        self.training = training
        self.nn_baseline = nn_baseline

    

## 5. `infrastructure/replay_buffer.py`



In [26]:
!pygmentize cs285/infrastructure/replay_buffer.py

from cs285.infrastructure.utils import *


class ReplayBuffer(object):

    def __init__(self, max_size=1000000):

        self.max_size = max_size

        # store each rollout
        self.paths = []

        # store (concatenated) component arrays from each rollout
        self.obs = None
        self.acs = None
        self.rews = None
        self.next_obs = None
        self.terminals = None

    def __len__(self):
        if self.obs:
            return self.obs.shape[0]
        else:
            return 0

    def add_rollouts(self, paths, concat_rew=True):

        # add new rollouts into our list of rollouts
        for path in paths:
            self.paths.append(path)

        # convert new rollouts into their component arrays, and append them onto
        # our arrays
        observations, actions, rewards, next_observations, terminals = (
            convert_listofrollouts(paths, concat_rew))

        if self.obs is None:
            self.obs = observations[-self.max_size:

## 7. `infrastructure/pytorch_util.py`

In [27]:
!pygmentize cs285/infrastructure/pytorch_util.py

from typing import Union

import torch
from torch import nn

Activation = Union[str, nn.Module]


_str_to_activation = {
    'relu': nn.ReLU(),
    'tanh': nn.Tanh(),
    'leaky_relu': nn.LeakyReLU(),
    'sigmoid': nn.Sigmoid(),
    'selu': nn.SELU(),
    'softplus': nn.Softplus(),
    'identity': nn.Identity(),
}


def build_mlp(
        input_size: int,
        output_size: int,
        n_layers: int,
        size: int,
        activation: Activation = 'tanh',
        output_activation: Activation = 'identity',
) -> nn.Module:
    """
        Builds a feedforward neural network

        arguments:
            n_layers: number of hidden layers
            size: dimension of each hidden layer
            activation: activation of each hidden layer

            input_size: size of the input layer
            output_size: size of the output layer
            output_activation: activation of the output layer

        returns:
            MLP (nn.Module)
    """
    if isinstance(activati

## Extra: `run_statistics.py` 

This one make it easier to explore hyperparameters running several simulations.


In [28]:
!pygmentize cs285/scripts/run_statistics.py

from run_hw1 import main as run_experiment
from os import listdir
from os.path import isfile, join
import json
import matplotlib.pyplot as plt
import numpy as np
# class stats():

def plot_results(logdirs, search_space):
    all_dicts = []
    for logdir in logdirs:
        for f in listdir(logdir):
            if str(f)[:7] == "metrics":
                # Metrics file
                with open(join(logdir, f)) as json_file:
                    data = dict(json.load(json_file))
                    all_dicts.append(data)

    # Merge everything in one dict with list as values
    results = {k: [dic[k] for dic in all_dicts] for k in all_dicts[0]}

    for metric, values in results.items():
        print('>>', metric, values)
        # From string to float
        values = [float(v) for v in values]
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.suptitle('{}'.format(metric))
        ax1.plot(search_space, values)
        
        ax1.plot(search_space, values,
                 l